# LSST Twilight Planner (Modular) — Notebook

This notebook demonstrates the use of the `twilight_planner_pkg` package to
generate twilight observing plans. The workflow is:

1. Set up imports and paths for the input SN catalog and output directory.
2. Build a fully specified `PlannerConfig` object with all optional parameters
   shown (defaults are used unless otherwise noted).
3. Run the planner and inspect the CSV outputs written to `OUTDIR`.


In [ ]:

# --- Imports and path setup ---
import os
import sys
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from twilight_planner_pkg.config import PlannerConfig
from twilight_planner_pkg.scheduler import plan_twilight_range_with_caps

# Edit these to your paths
catalog_csv = Path('../data/ATLAS_2021_to25_with_fakes_like_input.csv')
outdir = Path("../twilight_outputs")
outdir.mkdir(parents=True, exist_ok=True)

print('Catalog CSV:', catalog_csv)
print('Catalog exists:', catalog_csv.exists())
print('Output directory:', outdir)

# Legacy aliases used by some cells below
CSV_PATH = str(catalog_csv)
OUTDIR = str(outdir)


## Configure the planner

The next cells build a fully explicit `PlannerConfig`. A table after the configuration
lists **every available field and its current value**, so the notebook doubles as
documentation. Adjust values in the configuration cell to explore alternative
strategies or hardware assumptions. All text below is in English as requested.

### How to customise the run
- Update `catalog_csv` and `outdir` in the imports cell if your catalogue lives elsewhere.
- Edit the arguments passed to `PlannerConfig(...)` to override defaults.
- The table cell shows defaults for untouched parameters so you can audit the
  entire configuration before running the planner.

In [ ]:

# ---- Planner configuration (all fields shown, edit as needed) ----
# All parameters are explicitly listed with defaults or recommended values.
# Adjust values to explore different strategies; comments explain each group.

cfg = PlannerConfig(
    # -- Site ---------------------------------------------------------------
    lat_deg=-30.2446,        # Observatory latitude (deg)
    lon_deg=-70.7494,        # Observatory longitude (deg)
    height_m=2647.0,         # Elevation above sea level (m)

    # -- Filters and hardware ---------------------------------------------
    filters=['g','r','i','z'],    # Available filters (carousel)
    carousel_capacity=5,              # Carousel capacity (max filters mounted)
    filter_change_s=120.0,            # Time to change filters (s)
    readout_s=2.0,                    # CCD readout time (s)
    inter_exposure_min_s=15.0,        # Minimum time between exposures (s)
    filter_change_time_s=None,        # Legacy alias for filter_change_s
    readout_time_s=None,              # Legacy alias for readout_s
    exposure_by_filter={'u':30.0,'g':5.0,'r':5.0,'i':5.0,'z':5.0,'Y':5.0},  # Base exposure times (s)
    filters_per_visit_cap=1,          # Upper bound on filters used per visit
    auto_color_pairing=True,         # Automatically add opposite-colour filter when cap > 1
    start_filter=None,                # Start filter (None → first of filters)
    sun_alt_policy=[                  # Allowed filters vs Sun altitude ranges (deg)
        (-20.0,-16.0,['g','r','i','z']), # TODO: no Y band, need Z
        (-16.0,-12.0,['g','r','i','z']),
        (-12.0,-8.0,['r','i','z',"g"]),
        (-8.0,0.0,['r','i']),
    ],
    sun_alt_exposure_ladder=[],      # Optional exposure overrides per Sun-alt bucket
    filter_policy_use_best_time_alt=True,  # Use Sun alt at each target best_time (instead of window mid)

    # -- Slew model --------------------------------------------------------
    slew_small_deg=3.5,               # Small-angle definition (deg)
    slew_small_time_s=4.0,            # Time for small slew (s)
    slew_rate_deg_per_s=5.25,         # Rate for large slews (deg/s)
    slew_settle_s=1.0,                # Settling time after slew (s)

    # -- Moon --------------------------------------------------------------
    min_moon_sep_by_filter={          # Min Moon separation by filter (deg)
        'u':80.0,'g':50.0,'r':35.0,'i':30.0,'z':25.0,'Y':20.0,
    },
    require_single_time_for_all_filters=True,  # Enforce one best time for all filters of a visit (not used for now)

    # -- Time caps ---------------------------------------------------------
    min_alt_deg=30.0,                 # Min target altitude (deg)
    twilight_sun_alt_min_deg=-16.0,   # Sun altitude min for twilight window (deg)
    twilight_sun_alt_max_deg=-8.0,     # Sun altitude max for twilight window (deg)

    per_sn_cap_s=600.0,               # Max time per SN per night (s)
    morning_cap_s='auto',             # Morning twilight time cap (s or 'auto')
    evening_cap_s='auto',             # Evening twilight time cap (s or 'auto')

    # Optional manual local-time overrides for twilight windows
    morning_twilight=None,            # e.g. 'HH:MM' local start cut
    evening_twilight=None,            # e.g. 'HH:MM' local start cut

    twilight_step_min=2,              # Internal sampling step (minutes) for calculating like moon alt
    max_sn_per_night=999,    # Optional hard cap on number of SNe per night

    # -- Priority tracking -------------------------------------------------
    hybrid_detections=2,              # Detections for hybrid strategy
    hybrid_exposure_s=300.0,          # Exposure target for hybrid (s)
    lc_detections=60,                  # Detections for light-curve strategy
    lc_exposure_s=300.0,              # Exposure target for light-curve (s)
    priority_strategy='hybrid',       # 'hybrid' or 'lc' or 'unique_first'


    # -- Cadence -----------------------------------------------------------
    cadence_enable=True,              # Enable cadence gating and bonus
    cadence_per_filter=True,          # Track cadence per filter
    cadence_days_target=3.0,          # Target days between revisits
    cadence_jitter_days=0.5,         # Early revisit allowance (days)
    cadence_days_tolerance=1,       # KPI window around target (days)
    cadence_bonus_sigma_days=0.5,     # Width of due-soon bonus (days)
    cadence_bonus_weight=0.25,        # Weight of cadence bonus
    cadence_first_epoch_bonus_weight=0.0,  # Extra bonus for first epoch in a filter

    # -- Cosmology / colour tracking --------------------------------------
    cosmo_weight_by_filter={'g':1.25,'r':1.10,'i':1.00,'z':0.85,'Y':0.60},
    color_window_days=5.0,
    color_target_pairs=2,
    color_alpha=0.3,
    swap_cost_scale_color=0.6,
    swap_amortize_min=6,
    palette_rotation_days=4,
    # Palette order used as a tie-breaker; first-filter cycle can override
    palette_evening=['z','i','r','g'],
    palette_morning=['g','r','i','z'],
    max_swaps_per_window=4,
    first_epoch_color_boost=1.5,

    # -- Band diversity (per-filter balance) -----------------------------
    diversity_enable=True,
    diversity_target_per_filter=1,
    diversity_window_days=5.0,
    diversity_alpha=0.3,

    # -- First-filter cycle (per window/day) -----------------------------
    first_filter_cycle_enable=True,
    first_filter_cycle_morning=['g','r'],
    first_filter_cycle_evening=['z','i'],

    # -- Redshift prioritization -----------------------------------------
    redshift_boost_enable=True,       # Mild boost for low-z SNe
    redshift_low_ref=0.08,
    redshift_boost_max=5.0,
    redshift_column=None,             # Explicit redshift column in catalog (optional)

    # -- Low-z Ia special handling ----------------------------------------
    low_z_ia_markers=['ia','1','101'],
    low_z_ia_z_threshold= 0.08,
    low_z_ia_priority_multiplier=1.5,
    low_z_ia_cadence_days_target=2.5,
    low_z_ia_repeats_per_window=2,

    # -- Backfill relax cadence -------------------------------------------
    backfill_relax_cadence=True,     # Last-resort: relax cadence if time would go unused

    # -- Catalog pre-filtering --------------------------------------------
    only_ia=True,                    # Pre-filter to Ia-like types only

    # -- Photometry / sky --------------------------------------------------
    pixel_scale_arcsec=0.2,           # Arcsec per pixel
    zpt1s=None,                       # Optional per-filter zeropoint@1s
    k_m=None,                         # Optional per-filter extinction coefficient
    fwhm_eff=None,                    # Optional per-filter effective FWHM
    read_noise_e=6.0,                 # Read noise (e-)
    gain_e_per_adu=1.6,               # Gain (e-/ADU)
    zpt_err_mag=0.01,                 # Zeropoint error (mag)
    dark_sky_mag=None,                # Optional per-filter dark-sky mags
    twilight_delta_mag=2.5,           # Brightening of sky in twilight if no provider

    # -- Cosmology / peak-magnitude guardrails ----------------------------
    H0_km_s_Mpc=70.0,
    Omega_m=0.3,
    Omega_L=0.7,
    MB_absolute=-19.36,
    SALT2_alpha=0.14,
    SALT2_beta=3.1,
    Kcorr_approx_mag=0.0,
    Kcorr_approx_mag_by_filter={},
    peak_extra_bright_margin_mag=0.3, # Extra margin when using peak mags (mag)

    # -- SIMLIB ------------------------------------------------------------
    simlib_out=None,                  # Will be set below to write a SIMLIB file
    simlib_survey='LSST',
    simlib_filters='griz',           # SNANA filter string (we may set 'ugrizY')
    simlib_pixsize=0.2,
    simlib_npe_pixel_saturate=80000.0,
    simlib_photflag_saturate=2048,
    simlib_psf_unit='PIXEL',          # SNANA expects PSF1/PSF2/PSFRATIO in PIXEL

    # -- Miscellaneous -----------------------------------------------------
    typical_days_by_type={'Ia':30,'II-P':100,'II-L':70,'IIn':120,'IIb':70,'Ib':60,'Ic':60},
    default_typical_days=30,
    ra_col=None,
    dec_col=None,
    disc_col=None,
    name_col=None,
    type_col=None,

    # -- Scheduler-populated context (leave None) -------------------------
    current_mag_by_filter=None,
    current_alt_deg=None,
    current_mjd=None,
    current_redshift=None,
    sky_provider=None,

    # -- Optional per-target host galaxy context --------------------------
    current_host_mu_arcsec2_by_filter=None,
    current_host_mu_rest_arcsec2_by_filter=None,
    current_host_z=None,
    current_host_K_by_filter=None,
    current_host_point_mag_by_filter=None,
    current_host_point_frac=None,

    # -- Default host SB fallback when per-target inputs are missing ------
    use_default_host_sb=True,
    default_host_mu_arcsec2_by_filter={'u':22.0,'g':22.0,'r':22.0,'i':22.0,'z':22.0,'Y':22.0},
    default_host_mu_rest_arcsec2_by_filter={'u':22.8,'g':22.2,'r':21.7,'i':21.5,'z':21.4,'Y':21.3},
    default_host_kcorr_slope_by_filter={'u':0.35,'g':0.30,'r':0.25,'i':0.20,'z':0.20,'Y':0.15},

    # -- Backwards-compatibility options ----------------------------------
    allow_filter_changes_in_twilight=True,  # Permit changing filters within twilight

    # -- Discovery magnitude fallback (for saturation capping) -------------
    use_discovery_fallback=True,
    discovery_policy='atlas_priors',
    discovery_assumed_gr=0.0,
    discovery_margin_mag=0.2,
    discovery_color_priors_min={'u-g':0.3,'g-r':-0.25,'r-i':-0.15,'i-z':-0.10,'z-y':-0.05},
    discovery_color_priors_max={'u-g':1.0,'g-r':0.15,'r-i':0.25,'i-z':0.20,'z-y':0.30},
    discovery_non_ia_widen_mag=0.1,
    discovery_y_extra_margin_mag=0.25,
    discovery_atlas_linear={'c':{'alpha':0.0,'beta':-0.47}, 'o':{'alpha':0.0,'beta':0.26}},
    discovery_error_on_missing=True,
)

# Recommended SIMLIB header consistency for SNANA
cfg.simlib_filters = 'griz'  # include 'Y' if needed for upstream tools
cfg.simlib_pixsize = 0.200
cfg.simlib_psf_unit = 'PIXEL'

In [ ]:

import dataclasses
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

config_rows = []
for field in dataclasses.fields(PlannerConfig):
    value = getattr(cfg, field.name)
    config_rows.append({'field': field.name, 'type': str(field.type), 'value': repr(value)})

cfg_overview = pd.DataFrame(config_rows)
display(cfg_overview)

In [ ]:
# Use SNANA central-pixel fraction (Taylor expansion) for parity
from twilight_planner_pkg.photom_rubin import PhotomConfig
phot_defaults = PhotomConfig(
    pixel_scale_arcsec=cfg.pixel_scale_arcsec,
    gain_e_per_adu=cfg.gain_e_per_adu,
    npe_pixel_saturate=cfg.simlib_npe_pixel_saturate,
    npe_pixel_warn_nonlinear=int(0.8 * cfg.simlib_npe_pixel_saturate),
    use_snana_fA=True
)


In [ ]:
from functools import lru_cache
from twilight_planner_pkg import astro_utils as AU

@lru_cache(maxsize=4096)
def _peak_mag_cached(z: float, band: str) -> float:
    return AU.peak_mag_from_redshift(
        float(z), str(band).lower(),
        MB=cfg.MB_absolute, alpha=cfg.SALT2_alpha, beta=cfg.SALT2_beta,
        H0=cfg.H0_km_s_Mpc, Om=cfg.Omega_m, Ol=cfg.Omega_L,
        K_approx=cfg.Kcorr_approx_mag_by_filter.get(str(band).lower(), 0.0),
    )

# Route internal calls through the cache
AU.peak_mag_from_redshift = _peak_mag_cached


In [ ]:
# Diagnose catalog magnitude mapping for discovery fallback
from twilight_planner_pkg.examine import diagnose_mag_mapping
band_to_col, maglike_cols, preview = diagnose_mag_mapping(str(catalog_csv), cfg)
print('Band→column mapping:', band_to_col)
print('Columns containing "mag":', maglike_cols[:10])
display(preview)



## Run the planner

`plan_twilight_range_with_caps` is invoked below.  It reads the SN catalog from
`catalog_csv` (aliased as `CSV_PATH` for legacy cells), writes output CSVs to `outdir`
/ `OUTDIR`, and prints progress information.
Two files are produced for each run:

- `lsst_twilight_plan_<run_label>_*.csv` — per-SN schedule
- `lsst_twilight_summary_<run_label>_*.csv` — per-night summary


In [ ]:
start_date = '2023-01-01'
end_date = '2024-01-01'

# Mirror legacy variable names for later cells
START_DATE = start_date
END_DATE = end_date

# Enable SIMLIB output (SNANA expects PSF_UNIT=PIXEL)
cfg.simlib_out = str((outdir / f"twilight_{START_DATE}_to_{END_DATE}_single5s.simlib").resolve())

perSN_df, nights_df = plan_twilight_range_with_caps(
    csv_path=str(catalog_csv),
    outdir=str(outdir),
    start_date=start_date,
    end_date=end_date,
    cfg=cfg,
    run_label='hybrid',
    verbose=True
)

In [ ]:
perSN_df.head()

In [ ]:
# Show the nightly summary
nights_df


## Inspect outputs on disk


In [ ]:
import glob
import os

files = sorted(glob.glob(os.path.join(OUTDIR, "lsst_twilight_*.csv")))
files

In [ ]:
# Diagnose per-visit saturation contributions
from twilight_planner_pkg.examine import build_saturation_df
saturation_df = build_saturation_df(perSN_df, CSV_PATH, cfg)
# Save and show a compact view
sat_path = os.path.join(OUTDIR, f'saturation_{START_DATE}_to_{END_DATE}.csv')
saturation_df.to_csv(sat_path, index=False)
cols = ['SN','filter','t_exp_s','t_exp_s_base','src_mag','sky_mag_arcsec2','host_mu_arcsec2',
        'e_src','e_sky','e_host','total_e','sat_limit','sat_guard_applied','warn_nonlinear',
        'over_sat','over_sat_base']
display(saturation_df[cols])
print('Wrote:', sat_path)



In [ ]:
print("Filters used:", cfg.filters)
print("SIMLIB sat (NPE_PIXEL_SATURATE):", cfg.simlib_npe_pixel_saturate)
print("PSF_UNIT:", cfg.simlib_psf_unit)

# If SIMLIB was written, glance at header lines
if cfg.simlib_out:
    with open(cfg.simlib_out, "r") as fp:
        for i, line in enumerate(fp):
            if i > 25:
                break
            print(line.rstrip())
